In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import initialize_agent,AgentType
from langchain_core.tools import tool

# from langchain.chains import SequentialChain


# Load environment variables from the .env file
load_dotenv()

os.getenv("GROQ_API_KEY")



'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'

In [5]:
tools1=[
  {
      "tool_name": "who_am_i",
      "tool_description": "Returns the id of the current user",
      "args": [],
      "output": {
          "arg_type": "str",
          "is_array": False,
          "is_required": True
      }
  },
  {
      "tool_name": "get_sprint_id",
      "tool_description": "Returns the ID\nof the current\nsprint",
      "args": [],
      "output": {
          "arg_type": "str",
          "is_array": False,
          "is_required": True
      }
  },
  {
    "tool_name": "works_list",
    "tool_description": "Returns a list of work items matching the request",
    "args": [
    {
    "arg_name": "applies_to_part",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "created_by",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "issue_priority",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "issue.rev_orgs",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "limit",
    "arg_type": "int",
    "is_array": False,
    "is_required": False
    },
    {
    "arg_name": "owned_by",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "stage_name",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "ticket_need_response",
    "arg_type": "boolean",
    "is_array": False,
    "is_required": False
    },
    {
    "arg_name": "ticket_rev_org",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "ticket_severity",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "ticket_source_channel",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    },
    {
    "arg_name": "type",
    "arg_type": "str",
    "is_array": True,
    "is_required": False
    }
    ],
    "output": {
    "arg_type": "any",
    "is_array": False,
    "is_required": True
    }
    },
    {
    "tool_name": "summarize_objects",
    "tool_description": "Summarizes a list of objects. The logic of how to summarize a particular object type is an internal implementation detail.",
    "args": [
    {
    "arg_name": "objects",
    "arg_type": "any",
    "is_array": True,
    "is_required": True
    }
    ],
    "output": {
    "arg_type": "any",
    "is_array": False,
    "is_required": True
    }
    },
    {
    "tool_name": "prioritize_objects",
    "tool_description": "Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail",
    "args": [
    {
    "arg_name": "objects",
    "arg_type": "any",
    "is_array": False,
    "is_required": False
    }
    ],
    "output": {
    "arg_type": "any",
    "is_array": True,
    "is_required": True
    }
}]

# llm = ChatGroq(
# model="llama-3.1-8b-instant",
# temperature=0.3,
# max_tokens=None,
# timeout=None,
# max_retries=2,
# tools=tools,
# tool_choice="auto",
# )

In [6]:
import json
from groq import Groq
import os

# Initialize Groq client
client = Groq()




In [14]:
import instructor
import json
from pydantic import BaseModel
from typing import List

class Argument(BaseModel):
    argument_name: str
    argument_value: List[str]

class Tool(BaseModel):
    tool_name: str
    arguments: List[Argument]
    
class ResponseModel(BaseModel):
    tool_calls: list[Tool]
    
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON) 

def run_conversation(user_prompt,tools):
    tools=tools
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": f"""You are an assistant that can use tools. You have access to the following tool: {tools}.go through each argument provided properly and ive every needed arg
            when you need output from of i th previous tool for argument write "$$PREV[i]" in the argument .order the function calling properly for the task.
            """
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    # Make the Groq API call
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        response_model=ResponseModel,
        messages=messages,
        temperature=0.7,
        max_tokens=1000,
    )

    return response.tool_calls

# Example usage

def output(query,tools):
    tool_calls = run_conversation(query,tools)
    # print(tool_calls)
    
    output=[] 
    for i in tool_calls:
        output.append(i.json())
        
    return output    

user_prompt="Summarize tickets from ’support’ channel"   
output(user_prompt,tools1)    
    
    

['{"tool_name":"works_list","arguments":[{"argument_name":"ticket_source_channel","argument_value":["support"]}]}',
 '{"tool_name":"summarize_objects","arguments":[{"argument_name":"objects","argument_value":["$$PREV[0]"]}]}']

CREATIVITY